In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler

tqdm.pandas(desc="my bar!")
pd.set_option("max_columns", 100)
pd.set_option("max_rows", 100)

def disp_full(x, drows=False, dcols=True):
    if drows:
        pd.set_option('display.max_rows', x.shape[0])
    if dcols:
        pd.set_option('display.max_columns', x.shape[1])
    display(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')


def wlogloss(targets, preds):
    target_cols = [
        "any",
        "epidural",
        "subdural",
        "subarachnoid",
        "intraventricular",
        "intraparenchymal",
    ]
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col + "_pred"])
        if col == "any":
            res += log_loss(targets[col], preds[col + "_pred"])
    res /= 7
    return res


def make_feats(df):
    df["ImagePositionPatient_2"] = df["ImagePositionPatient"].progress_apply(
        lambda x: x[2]
    )
    df = df.merge(
        df.groupby(["StudyInstanceUID"])["ImagePositionPatient_2"]
        .agg(position_min="min", position_max="max")
        .reset_index(),
        on="StudyInstanceUID",
    )
    df["position"] = (df["ImagePositionPatient_2"] - df["position_min"]) / (
        df["position_max"] - df["position_min"]
    )
    res = df.sort_values(by=["StudyInstanceUID", "position"])
    return res


def pred_agg1_train(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if c != "any_pred_sasaki_se_resnext_410":
            del tmp["StudyInstanceUID"]
        else:
            print(c)
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg1_test(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if "any_pred" not in c:
            del tmp["StudyInstanceUID"]
        else:
            print(c)
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg2(df):
    pred_cols = [column for column in df.columns if "pred" in column]
    a1 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(3, min_periods=1, center=True)
        .mean()
        .values
    )
    a2 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(5, min_periods=1, center=True)
        .mean()
        .values
    )
    a3 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(1, min_periods=1, center=True)
        .mean()
        .values
    )
    new_feats1 = pd.DataFrame(a1, columns=[c + "_3roll" for c in pred_cols])
    new_feats2 = pd.DataFrame(a2, columns=[c + "_5roll" for c in pred_cols])
    new_feats3 = pd.DataFrame(a1 - a3, columns=[c + "_3rolldiff" for c in pred_cols])
    new_feats4 = pd.DataFrame(a2 - a3, columns=[c + "_5rolldiff" for c in pred_cols])
    new_feats5 = pd.DataFrame(a1 / a3, columns=[c + "_3rolldiv" for c in pred_cols])
    new_feats6 = pd.DataFrame(a2 / a3, columns=[c + "_5rolldiv" for c in pred_cols])
    new_feats1.index = df.index
    new_feats2.index = df.index
    new_feats3.index = df.index
    new_feats4.index = df.index
    new_feats5.index = df.index
    new_feats6.index = df.index
    df = pd.concat(
        [df, new_feats1, new_feats2, new_feats3, new_feats4, new_feats5, new_feats6],
        axis=1,
    )
    return df

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
train = pd.read_pickle('stage2_data/train_raw.pkl')
train = make_feats(train)
test = pd.read_pickle("stage2_data/test_raw.pkl")
test = make_feats(test)

my bar!: 100%|██████████| 121232/121232 [00:00<00:00, 472498.82it/s]


In [3]:
target_cols = [
    "any",
    "epidural",
    "subdural",
    "subarachnoid",
    "intraventricular",
    "intraparenchymal",
]
models = [
    "sasaki_se_resnext101_mixup_410_with_imagenet_norm",
    "sasaki_se_resnext_410",
    "sasaki_senet154_customlabels",
    "shimakoshi_densenet_adj_prediction",
    "shimakoshi_seresnext50_label_smoothing",
    "shimakoshi_seresnext50_label_smoothing_not_any",
    "tsukiyama_inception_resnet_v2",
    "tsukiyama_se_resnext",
    "tsukiyama_xception",
    "tsukiyama_inceptionv4",
]
shimakoshi_label_dict = {
    "pred_score0": "any_pred",
    "pred_score1": "epidural_pred",
    "pred_score2": "subdural_pred",
    "pred_score3": "subarachnoid_pred",
    "pred_score4": "intraventricular_pred",
    "pred_score5": "intraparenchymal_pred"
}

n_tta = 5
n_folds = 5

In [4]:
# train

model_dfs = []

for model in tqdm(models):
    df_all = []
    for n_fold in range(n_folds):
        df = pd.read_pickle(f"stage2_outputs/{model}/fold{n_fold}_ep2_valid_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
            tmp = pd.DataFrame(tmp)
            tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
            tmp["ID"] = df[0]["ids"]
        else:
            tmp = df[["ID", ]+list(shimakoshi_label_dict.keys())].rename(columns=shimakoshi_label_dict)
        tmp["folds"] = n_fold
            
        label_df = pd.read_pickle(f"stage2_outputs/tsukiyama_inceptionv4/fold{n_fold}_ep2_valid_tta5.pkl")
        target = label_df[0]["targets"]
        tmp2 = pd.DataFrame(target)
        tmp2.columns = [tar_col + "_true" for tar_col in target_cols]
        tmp2["ID"] = label_df[0]["ids"]
        assert len(set(tmp["ID"].tolist())-set(tmp2["ID"].tolist())) == 0
        df_all.append(pd.merge(tmp, tmp2, on="ID", how="left"))
    df_all = pd.concat(df_all).reset_index(drop=True)
    model_dfs.append(df_all)
                       
columns = list(model_dfs[0].columns)
print(columns)
columns.remove("ID")
for i, model_df in enumerate(model_dfs):
    rename_dict = {}
    for column in columns:
        rename_dict[column] = column+"_{}".format(i)
    if i == 0:
        df = model_df.rename(columns=rename_dict)
    else:
        df = pd.merge(df, model_df.rename(columns=rename_dict), on="ID", how="outer")
        
"""
model_dfs = []
for model in tqdm(models):
    df_all = []
    n_tta = 5
    for n_fold in range(5):
        df = pd.read_pickle(f"stage2_outputs/{model}/fold{n_fold}_ep2_valid_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
        else:
            tmp = df["outputs"]
        tmp = pd.DataFrame(tmp)
        tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
        if "shimakoshi" not in model:
            tmp["ID"] = df[0]["ids"]
            target = df[0]["targets"]
        else:
            tmp["ID"] = df["ids"]
            target = df["targets"]
        tmp["folds"] = n_fold
        if model == "sasaki_senet154_customlabels":
            target[target<0.5] = 0
        tmp2 = pd.DataFrame(target)
        tmp2.columns = [tar_col + "_true" for tar_col in target_cols]
        df_all.append(pd.concat([tmp, tmp2], axis=1))
    df_all = pd.concat(df_all)
    model_dfs.append(df_all)

columns = list(model_dfs[0].columns)
print(columns)
columns.remove("ID")
for i, model_df in enumerate(model_dfs):
    rename_dict = {}
    for column in columns:
        rename_dict[column] = column+"_{}".format(i)
    if i == 0:
        df = model_df.rename(columns=rename_dict)
    else:
        df = pd.merge(df, model_df.rename(columns=rename_dict), on="ID", how="outer")
"""

100%|██████████| 10/10 [00:50<00:00,  5.04s/it]


['any_pred', 'epidural_pred', 'subdural_pred', 'subarachnoid_pred', 'intraventricular_pred', 'intraparenchymal_pred', 'ID', 'folds', 'any_true', 'epidural_true', 'subdural_true', 'subarachnoid_true', 'intraventricular_true', 'intraparenchymal_true']


'\nmodel_dfs = []\nfor model in tqdm(models):\n    df_all = []\n    n_tta = 5\n    for n_fold in range(5):\n        df = pd.read_pickle(f"stage2_outputs/{model}/fold{n_fold}_ep2_valid_tta5.pkl")\n        if "shimakoshi" not in model:\n            tmp = np.zeros([len(df[0]["ids"]), 6])\n            for i in range(n_tta):\n                tmp += df[i]["outputs"] / n_tta\n        else:\n            tmp = df["outputs"]\n        tmp = pd.DataFrame(tmp)\n        tmp.columns = [tar_col + "_pred" for tar_col in target_cols]\n        if "shimakoshi" not in model:\n            tmp["ID"] = df[0]["ids"]\n            target = df[0]["targets"]\n        else:\n            tmp["ID"] = df["ids"]\n            target = df["targets"]\n        tmp["folds"] = n_fold\n        if model == "sasaki_senet154_customlabels":\n            target[target<0.5] = 0\n        tmp2 = pd.DataFrame(target)\n        tmp2.columns = [tar_col + "_true" for tar_col in target_cols]\n        df_all.append(pd.concat([tmp, tmp2], ax

In [57]:
#pred_columns = [c for c in df.columns if "pred" in c]
#df[pred_columns] = df[pred_columns].clip(1e-15, 1-1e-15)

In [5]:
for i, row in df[df.isnull().any(axis=1)].iterrows():
    for column in columns:
        print(i)
        tmp = [c for c in df.columns if column in c]
        nan_columns = []
        for j, t in enumerate(row[tmp].isnull()):
            if t:
                nan_columns.append(tmp[j])
        tmp = row[tmp].mean()
        df.loc[i, nan_columns] = tmp

In [6]:
n_model = len(model_dfs)
model_dfs = []
for i in range(n_model):
    tmp_columns = [c for c in df.columns if "_{}".format(i) in c]
    df_all = df[tmp_columns+["ID"]]
    rename_dict = {}
    for tmp_column in tmp_columns:
        rename_dict[tmp_column] = tmp_column.replace("_{}".format(i), "").replace("_true", "")
    df_all = df_all.rename(columns=rename_dict)
    model_dfs.append(df_all)
    
for i, model_df in enumerate(model_dfs):
    rename_dict = {}
    for target_col in target_cols:
        rename_dict[target_col+"_pred"] = target_col + "_pred_" + models[i]
    if i == 0:
        df_all = model_df.rename(columns=rename_dict)
    else:
        c = model_df.columns.drop(target_cols+["folds", ])
        df_all = pd.merge(df_all, model_df[c].rename(columns=rename_dict), on="ID", how="left")
print(df_all.columns)

Index(['any_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'epidural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subdural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subarachnoid_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraventricular_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraparenchymal_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'folds', 'any', 'epidural', 'subdural', 'subarachnoid',
       'intraventricular', 'intraparenchymal', 'ID',
       'any_pred_sasaki_se_resnext_410', 'epidural_pred_sasaki_se_resnext_410',
       'subdural_pred_sasaki_se_resnext_410',
       'subarachnoid_pred_sasaki_se_resnext_410',
       'intraventricular_pred_sasaki_se_resnext_410',
       'intraparenchymal_pred_sasaki_se_resnext_410',
       'any_pred_sasaki_senet154_customlabels',
       'epidural_pred_sasaki_senet154_customlabels',
       'subdural_pred_sasaki_senet154_cu

In [7]:
df_all = df_all.merge(train, on='ID')
if "position" in df_all.columns:
    df_all = df_all.sort_values(['StudyInstanceUID', 'position'])
else:
    df_all = df_all.sort_values(['StudyInstanceUID'])
    
df_all = pred_agg1_train(df_all)
df_all = pred_agg2(df_all)

any_pred_sasaki_se_resnext_410


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in true_divide


In [43]:
#with open("df_all.pkl", "wb") as f:
#    pickle.dump(df_all, f)
#with open("df_all.pkl", "rb") as f:
#    df_all = pickle.load(f)

In [8]:
# test

model_df_list_test = []
for model in tqdm(models):
    df_all_test = []
    for n_fold in range(n_folds):
        df = pd.read_pickle(f"stage2_outputs/{model}/fold{n_fold}_ep2_test_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
            tmp = pd.DataFrame(tmp)
            tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
            tmp["ID"] = df[0]["ids"]
        else:
            tmp = df[["ID", ]+list(shimakoshi_label_dict.keys())].rename(columns=shimakoshi_label_dict)
        tmp["folds"] = n_fold
        df_all_test.append(tmp)
    model_df_list_test.append(df_all_test)


100%|██████████| 10/10 [02:18<00:00, 13.85s/it]


In [6]:
"""
# test

model_df_list_test = []
for model in tqdm(models):
    df_all_test = []
    for n_fold in range(n_folds):
        df = pd.read_pickle(f"stage2_outputs/{model}/fold{n_fold}_ep2_test_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
        else:
            tmp = df["outputs"]
        tmp = pd.DataFrame(tmp)
        tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
        if "shimakoshi" not in model:
            tmp["ID"] = df[n_fold]["ids"]
        else:
            tmp["ID"] = df["ids"]
        tmp["folds"] = n_fold
        pred_columns = [c for c in tmp.columns if "pred" in c]
        tmp[pred_columns] = tmp[pred_columns].clip(1e-15, 1-1e-15)
        df_all_test.append(tmp)
    model_df_list_test.append(df_all_test)
"""

 50%|█████     | 3/6 [00:06<00:06,  2.18s/it]


KeyError: 'outputs'

In [9]:
columns_test = list(model_df_list_test[0][0].columns)
columns_test.remove("ID")
print(columns_test)
averaged_df_list_test = []
for j in range(len(model_df_list_test[0])):
    for i in range(len(model_df_list_test)):
        rename_dict = {}
        for column in columns_test:
            rename_dict[column] = column+"_{}".format(i)
        if i == 0:
            tmp_df = model_df_list_test[i][j].rename(columns=rename_dict)
        else:
            #tmp_df = pd.merge(tmp_df, model_df_list_test[i][j].drop(columns=["folds"]), on="ID", how="outer")
            tmp_df = pd.merge(tmp_df, model_df_list_test[i][j].rename(columns=rename_dict), on="ID", how="outer")
    averaged_df_list_test.append(tmp_df)

['any_pred', 'epidural_pred', 'subdural_pred', 'subarachnoid_pred', 'intraventricular_pred', 'intraparenchymal_pred', 'folds']


In [10]:
for j, averaged_df in enumerate(averaged_df_list_test):
    for i, row in averaged_df[averaged_df.isnull().any(axis=1)].iterrows():
        print(i)
        for column in columns_test:
            print(j, i, column)
            tmp = [c for c in averaged_df.columns if column in c]
            nan_columns = []
            for k, t in enumerate(row[tmp].isnull()):
                if t:
                    nan_columns.append(tmp[k])
            tmp = row[tmp].mean()
            averaged_df_list_test[j].loc[i, nan_columns] = tmp
    print(np.sum(averaged_df_list_test[0].isnull().all(axis=1)))
    averaged_df[[c for c in averaged_df.columns if c != "ID"]] = averaged_df[[c for c in averaged_df.columns if c != "ID"]].fillna(1e-15)
    print(np.sum(averaged_df_list_test[0].isnull().all(axis=1)))

61
0 61 any_pred
0 61 epidural_pred
0 61 subdural_pred
0 61 subarachnoid_pred
0 61 intraventricular_pred
0 61 intraparenchymal_pred
0 61 folds
71
0 71 any_pred
0 71 epidural_pred
0 71 subdural_pred
0 71 subarachnoid_pred
0 71 intraventricular_pred
0 71 intraparenchymal_pred
0 71 folds
133
0 133 any_pred
0 133 epidural_pred
0 133 subdural_pred
0 133 subarachnoid_pred
0 133 intraventricular_pred
0 133 intraparenchymal_pred
0 133 folds
295
0 295 any_pred
0 295 epidural_pred
0 295 subdural_pred
0 295 subarachnoid_pred
0 295 intraventricular_pred
0 295 intraparenchymal_pred
0 295 folds
388
0 388 any_pred
0 388 epidural_pred
0 388 subdural_pred
0 388 subarachnoid_pred
0 388 intraventricular_pred
0 388 intraparenchymal_pred
0 388 folds
393
0 393 any_pred
0 393 epidural_pred
0 393 subdural_pred
0 393 subarachnoid_pred
0 393 intraventricular_pred
0 393 intraparenchymal_pred
0 393 folds
530
0 530 any_pred
0 530 epidural_pred
0 530 subdural_pred
0 530 subarachnoid_pred
0 530 intraventricular_pred

0 7367 subarachnoid_pred
0 7367 intraventricular_pred
0 7367 intraparenchymal_pred
0 7367 folds
7639
0 7639 any_pred
0 7639 epidural_pred
0 7639 subdural_pred
0 7639 subarachnoid_pred
0 7639 intraventricular_pred
0 7639 intraparenchymal_pred
0 7639 folds
7678
0 7678 any_pred
0 7678 epidural_pred
0 7678 subdural_pred
0 7678 subarachnoid_pred
0 7678 intraventricular_pred
0 7678 intraparenchymal_pred
0 7678 folds
7863
0 7863 any_pred
0 7863 epidural_pred
0 7863 subdural_pred
0 7863 subarachnoid_pred
0 7863 intraventricular_pred
0 7863 intraparenchymal_pred
0 7863 folds
7967
0 7967 any_pred
0 7967 epidural_pred
0 7967 subdural_pred
0 7967 subarachnoid_pred
0 7967 intraventricular_pred
0 7967 intraparenchymal_pred
0 7967 folds
8074
0 8074 any_pred
0 8074 epidural_pred
0 8074 subdural_pred
0 8074 subarachnoid_pred
0 8074 intraventricular_pred
0 8074 intraparenchymal_pred
0 8074 folds
8311
0 8311 any_pred
0 8311 epidural_pred
0 8311 subdural_pred
0 8311 subarachnoid_pred
0 8311 intraventricul

0 15758 intraparenchymal_pred
0 15758 folds
15918
0 15918 any_pred
0 15918 epidural_pred
0 15918 subdural_pred
0 15918 subarachnoid_pred
0 15918 intraventricular_pred
0 15918 intraparenchymal_pred
0 15918 folds
16112
0 16112 any_pred
0 16112 epidural_pred
0 16112 subdural_pred
0 16112 subarachnoid_pred
0 16112 intraventricular_pred
0 16112 intraparenchymal_pred
0 16112 folds
16153
0 16153 any_pred
0 16153 epidural_pred
0 16153 subdural_pred
0 16153 subarachnoid_pred
0 16153 intraventricular_pred
0 16153 intraparenchymal_pred
0 16153 folds
16246
0 16246 any_pred
0 16246 epidural_pred
0 16246 subdural_pred
0 16246 subarachnoid_pred
0 16246 intraventricular_pred
0 16246 intraparenchymal_pred
0 16246 folds
16274
0 16274 any_pred
0 16274 epidural_pred
0 16274 subdural_pred
0 16274 subarachnoid_pred
0 16274 intraventricular_pred
0 16274 intraparenchymal_pred
0 16274 folds
16327
0 16327 any_pred
0 16327 epidural_pred
0 16327 subdural_pred
0 16327 subarachnoid_pred
0 16327 intraventricular_pre

0 24426 folds
24670
0 24670 any_pred
0 24670 epidural_pred
0 24670 subdural_pred
0 24670 subarachnoid_pred
0 24670 intraventricular_pred
0 24670 intraparenchymal_pred
0 24670 folds
24826
0 24826 any_pred
0 24826 epidural_pred
0 24826 subdural_pred
0 24826 subarachnoid_pred
0 24826 intraventricular_pred
0 24826 intraparenchymal_pred
0 24826 folds
24892
0 24892 any_pred
0 24892 epidural_pred
0 24892 subdural_pred
0 24892 subarachnoid_pred
0 24892 intraventricular_pred
0 24892 intraparenchymal_pred
0 24892 folds
24933
0 24933 any_pred
0 24933 epidural_pred
0 24933 subdural_pred
0 24933 subarachnoid_pred
0 24933 intraventricular_pred
0 24933 intraparenchymal_pred
0 24933 folds
24949
0 24949 any_pred
0 24949 epidural_pred
0 24949 subdural_pred
0 24949 subarachnoid_pred
0 24949 intraventricular_pred
0 24949 intraparenchymal_pred
0 24949 folds
24954
0 24954 any_pred
0 24954 epidural_pred
0 24954 subdural_pred
0 24954 subarachnoid_pred
0 24954 intraventricular_pred
0 24954 intraparenchymal_pre

0 31923 folds
32159
0 32159 any_pred
0 32159 epidural_pred
0 32159 subdural_pred
0 32159 subarachnoid_pred
0 32159 intraventricular_pred
0 32159 intraparenchymal_pred
0 32159 folds
32234
0 32234 any_pred
0 32234 epidural_pred
0 32234 subdural_pred
0 32234 subarachnoid_pred
0 32234 intraventricular_pred
0 32234 intraparenchymal_pred
0 32234 folds
32370
0 32370 any_pred
0 32370 epidural_pred
0 32370 subdural_pred
0 32370 subarachnoid_pred
0 32370 intraventricular_pred
0 32370 intraparenchymal_pred
0 32370 folds
32411
0 32411 any_pred
0 32411 epidural_pred
0 32411 subdural_pred
0 32411 subarachnoid_pred
0 32411 intraventricular_pred
0 32411 intraparenchymal_pred
0 32411 folds
32412
0 32412 any_pred
0 32412 epidural_pred
0 32412 subdural_pred
0 32412 subarachnoid_pred
0 32412 intraventricular_pred
0 32412 intraparenchymal_pred
0 32412 folds
32422
0 32422 any_pred
0 32422 epidural_pred
0 32422 subdural_pred
0 32422 subarachnoid_pred
0 32422 intraventricular_pred
0 32422 intraparenchymal_pre

...

121004
4 121004 any_pred
4 121004 epidural_pred
4 121004 subdural_pred
4 121004 subarachnoid_pred
4 121004 intraventricular_pred
4 121004 intraparenchymal_pred
4 121004 folds
121005
4 121005 any_pred
4 121005 epidural_pred
4 121005 subdural_pred
4 121005 subarachnoid_pred
4 121005 intraventricular_pred
4 121005 intraparenchymal_pred
4 121005 folds
121006
4 121006 any_pred
4 121006 epidural_pred
4 121006 subdural_pred
4 121006 subarachnoid_pred
4 121006 intraventricular_pred
4 121006 intraparenchymal_pred
4 121006 folds
121007
4 121007 any_pred
4 121007 epidural_pred
4 121007 subdural_pred
4 121007 subarachnoid_pred
4 121007 intraventricular_pred
4 121007 intraparenchymal_pred
4 121007 folds
121008
4 121008 any_pred
4 121008 epidural_pred
4 121008 subdural_pred
4 121008 subarachnoid_pred
4 121008 intraventricular_pred
4 121008 intraparenchymal_pred
4 121008 folds
121009
4 121009 any_pred
4 121009 epidural_pred
4 121009 subdural_pred
4 121009 subarachnoid_pred
4 121009 intraventricular_p

4 121053 intraventricular_pred
4 121053 intraparenchymal_pred
4 121053 folds
121054
4 121054 any_pred
4 121054 epidural_pred
4 121054 subdural_pred
4 121054 subarachnoid_pred
4 121054 intraventricular_pred
4 121054 intraparenchymal_pred
4 121054 folds
121055
4 121055 any_pred
4 121055 epidural_pred
4 121055 subdural_pred
4 121055 subarachnoid_pred
4 121055 intraventricular_pred
4 121055 intraparenchymal_pred
4 121055 folds
121056
4 121056 any_pred
4 121056 epidural_pred
4 121056 subdural_pred
4 121056 subarachnoid_pred
4 121056 intraventricular_pred
4 121056 intraparenchymal_pred
4 121056 folds
121057
4 121057 any_pred
4 121057 epidural_pred
4 121057 subdural_pred
4 121057 subarachnoid_pred
4 121057 intraventricular_pred
4 121057 intraparenchymal_pred
4 121057 folds
121058
4 121058 any_pred
4 121058 epidural_pred
4 121058 subdural_pred
4 121058 subarachnoid_pred
4 121058 intraventricular_pred
4 121058 intraparenchymal_pred
4 121058 folds
121059
4 121059 any_pred
4 121059 epidural_pred


4 121102 folds
121103
4 121103 any_pred
4 121103 epidural_pred
4 121103 subdural_pred
4 121103 subarachnoid_pred
4 121103 intraventricular_pred
4 121103 intraparenchymal_pred
4 121103 folds
121104
4 121104 any_pred
4 121104 epidural_pred
4 121104 subdural_pred
4 121104 subarachnoid_pred
4 121104 intraventricular_pred
4 121104 intraparenchymal_pred
4 121104 folds
121105
4 121105 any_pred
4 121105 epidural_pred
4 121105 subdural_pred
4 121105 subarachnoid_pred
4 121105 intraventricular_pred
4 121105 intraparenchymal_pred
4 121105 folds
121106
4 121106 any_pred
4 121106 epidural_pred
4 121106 subdural_pred
4 121106 subarachnoid_pred
4 121106 intraventricular_pred
4 121106 intraparenchymal_pred
4 121106 folds
121107
4 121107 any_pred
4 121107 epidural_pred
4 121107 subdural_pred
4 121107 subarachnoid_pred
4 121107 intraventricular_pred
4 121107 intraparenchymal_pred
4 121107 folds
121108
4 121108 any_pred
4 121108 epidural_pred
4 121108 subdural_pred
4 121108 subarachnoid_pred
4 121108 int

4 121151 intraventricular_pred
4 121151 intraparenchymal_pred
4 121151 folds
121152
4 121152 any_pred
4 121152 epidural_pred
4 121152 subdural_pred
4 121152 subarachnoid_pred
4 121152 intraventricular_pred
4 121152 intraparenchymal_pred
4 121152 folds
121153
4 121153 any_pred
4 121153 epidural_pred
4 121153 subdural_pred
4 121153 subarachnoid_pred
4 121153 intraventricular_pred
4 121153 intraparenchymal_pred
4 121153 folds
121154
4 121154 any_pred
4 121154 epidural_pred
4 121154 subdural_pred
4 121154 subarachnoid_pred
4 121154 intraventricular_pred
4 121154 intraparenchymal_pred
4 121154 folds
121155
4 121155 any_pred
4 121155 epidural_pred
4 121155 subdural_pred
4 121155 subarachnoid_pred
4 121155 intraventricular_pred
4 121155 intraparenchymal_pred
4 121155 folds
121156
4 121156 any_pred
4 121156 epidural_pred
4 121156 subdural_pred
4 121156 subarachnoid_pred
4 121156 intraventricular_pred
4 121156 intraparenchymal_pred
4 121156 folds
121157
4 121157 any_pred
4 121157 epidural_pred


4 121200 subdural_pred
4 121200 subarachnoid_pred
4 121200 intraventricular_pred
4 121200 intraparenchymal_pred
4 121200 folds
121201
4 121201 any_pred
4 121201 epidural_pred
4 121201 subdural_pred
4 121201 subarachnoid_pred
4 121201 intraventricular_pred
4 121201 intraparenchymal_pred
4 121201 folds
121202
4 121202 any_pred
4 121202 epidural_pred
4 121202 subdural_pred
4 121202 subarachnoid_pred
4 121202 intraventricular_pred
4 121202 intraparenchymal_pred
4 121202 folds
121203
4 121203 any_pred
4 121203 epidural_pred
4 121203 subdural_pred
4 121203 subarachnoid_pred
4 121203 intraventricular_pred
4 121203 intraparenchymal_pred
4 121203 folds
121204
4 121204 any_pred
4 121204 epidural_pred
4 121204 subdural_pred
4 121204 subarachnoid_pred
4 121204 intraventricular_pred
4 121204 intraparenchymal_pred
4 121204 folds
121205
4 121205 any_pred
4 121205 epidural_pred
4 121205 subdural_pred
4 121205 subarachnoid_pred
4 121205 intraventricular_pred
4 121205 intraparenchymal_pred
4 121205 fold

In [11]:
n_model = len(model_df_list_test)
model_df_list_test = [[] for i in range(n_model)]
for j, averaged_df in enumerate(averaged_df_list_test):
    for i in range(n_model):
        tmp_columns = [c for c in averaged_df.columns if "_{}".format(i) in c]
        df_all_test = averaged_df[tmp_columns+["ID"]]
        rename_dict = {}
        for tmp_column in tmp_columns:
            rename_dict[tmp_column] = tmp_column.replace("_{}".format(i), "").replace("_true", "")
        df_all_test = df_all_test.rename(columns=rename_dict)
        model_df_list_test[i].append(df_all_test)

In [12]:
for j, model_dfs_test in enumerate(model_df_list_test):
    for i, model_df in enumerate(model_dfs_test):
        rename_dict = {}
        for target_col in target_cols:
            rename_dict[target_col+"_pred"] = target_col + "_pred_" + models[j]
        model_df = model_df.rename(columns=rename_dict)
        model_df = model_df.merge(test, on="ID")
        if "position" in model_df.columns:
            model_df = model_df.sort_values(['StudyInstanceUID', 'position'])
        else:
            model_df = model_df.sort_values(['StudyInstanceUID'])
        print(model_df.columns)
        model_df = pred_agg1_test(model_df)
        model_df = pred_agg2(model_df)
        model_df_list_test[j][i] = model_df


Index(['any_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'epidural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subdural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subarachnoid_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraventricular_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraparenchymal_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'folds', 'ID', 'labels', 'n_label', 'BitsAllocated', 'BitsStored',
       'Columns', 'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean',

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in true_divide


Index(['any_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'epidural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subdural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subarachnoid_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraventricular_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraparenchymal_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'folds', 'ID', 'labels', 'n_label', 'BitsAllocated', 'BitsStored',
       'Columns', 'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean',

Index(['any_pred_sasaki_se_resnext_410', 'epidural_pred_sasaki_se_resnext_410',
       'subdural_pred_sasaki_se_resnext_410',
       'subarachnoid_pred_sasaki_se_resnext_410',
       'intraventricular_pred_sasaki_se_resnext_410',
       'intraparenchymal_pred_sasaki_se_resnext_410', 'folds', 'ID', 'labels',
       'n_label', 'BitsAllocated', 'BitsStored', 'Columns', 'HighBit',
       'ImageOrientationPatient', 'ImagePositionPatient', 'Modality',
       'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
       'PixelSpacing', 'RescaleIntercept', 'RescaleSlope', 'Rows',
       'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID',
       'StudyInstanceUID', 'WindowCenter', 'WindowWidth', 'raw_max', 'raw_min',
       'raw_mean', 'raw_diff', 'doctor_max', 'doctor_min', 'doctor_mean',
       'doctor_diff', 'custom_max', 'custom_min', 'custom_mean', 'custom_diff',
       'ImagePositionPatient_2', 'position_min', 'position_max', 'position'],
      dtype='object')
a

Index(['any_pred_shimakoshi_densenet_adj_prediction',
       'epidural_pred_shimakoshi_densenet_adj_prediction',
       'subdural_pred_shimakoshi_densenet_adj_prediction',
       'subarachnoid_pred_shimakoshi_densenet_adj_prediction',
       'intraventricular_pred_shimakoshi_densenet_adj_prediction',
       'intraparenchymal_pred_shimakoshi_densenet_adj_prediction', 'folds',
       'ID', 'labels', 'n_label', 'BitsAllocated', 'BitsStored', 'Columns',
       'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean', 'doctor_diff', 'custom_max',
       'custom_min', 'custom_mean', 'custom_diff', 'ImagePos

Index(['any_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'epidural_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'subdural_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'subarachnoid_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'intraventricular_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'intraparenchymal_pred_shimakoshi_seresnext50_label_smoothing_not_any',
       'folds', 'ID', 'labels', 'n_label', 'BitsAllocated', 'BitsStored',
       'Columns', 'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean', 'doctor_diff', 'c

Index(['any_pred_tsukiyama_inception_resnet_v2',
       'epidural_pred_tsukiyama_inception_resnet_v2',
       'subdural_pred_tsukiyama_inception_resnet_v2',
       'subarachnoid_pred_tsukiyama_inception_resnet_v2',
       'intraventricular_pred_tsukiyama_inception_resnet_v2',
       'intraparenchymal_pred_tsukiyama_inception_resnet_v2', 'folds', 'ID',
       'labels', 'n_label', 'BitsAllocated', 'BitsStored', 'Columns',
       'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean', 'doctor_diff', 'custom_max',
       'custom_min', 'custom_mean', 'custom_diff', 'ImagePositionPatient_2',
       'posit

Index(['any_pred_tsukiyama_xception', 'epidural_pred_tsukiyama_xception',
       'subdural_pred_tsukiyama_xception',
       'subarachnoid_pred_tsukiyama_xception',
       'intraventricular_pred_tsukiyama_xception',
       'intraparenchymal_pred_tsukiyama_xception', 'folds', 'ID', 'labels',
       'n_label', 'BitsAllocated', 'BitsStored', 'Columns', 'HighBit',
       'ImageOrientationPatient', 'ImagePositionPatient', 'Modality',
       'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
       'PixelSpacing', 'RescaleIntercept', 'RescaleSlope', 'Rows',
       'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID',
       'StudyInstanceUID', 'WindowCenter', 'WindowWidth', 'raw_max', 'raw_min',
       'raw_mean', 'raw_diff', 'doctor_max', 'doctor_min', 'doctor_mean',
       'doctor_diff', 'custom_max', 'custom_min', 'custom_mean', 'custom_diff',
       'ImagePositionPatient_2', 'position_min', 'position_max', 'position'],
      dtype='object')
any_pred_tsukiyama_

Index(['any_pred_tsukiyama_inceptionv4', 'epidural_pred_tsukiyama_inceptionv4',
       'subdural_pred_tsukiyama_inceptionv4',
       'subarachnoid_pred_tsukiyama_inceptionv4',
       'intraventricular_pred_tsukiyama_inceptionv4',
       'intraparenchymal_pred_tsukiyama_inceptionv4', 'folds', 'ID', 'labels',
       'n_label', 'BitsAllocated', 'BitsStored', 'Columns', 'HighBit',
       'ImageOrientationPatient', 'ImagePositionPatient', 'Modality',
       'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
       'PixelSpacing', 'RescaleIntercept', 'RescaleSlope', 'Rows',
       'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID',
       'StudyInstanceUID', 'WindowCenter', 'WindowWidth', 'raw_max', 'raw_min',
       'raw_mean', 'raw_diff', 'doctor_max', 'doctor_min', 'doctor_mean',
       'doctor_diff', 'custom_max', 'custom_min', 'custom_mean', 'custom_diff',
       'ImagePositionPatient_2', 'position_min', 'position_max', 'position'],
      dtype='object')
a

In [133]:
#with open("model_df_list_test.pkl", "wb") as f:
#    pickle.dump(model_df_list_test, f)
#with open("model_df_list_test.pkl", "rb") as f:
#    model_df_list_test = pickle.load(f)

In [13]:
#object colはone hot encoding

from sklearn.preprocessing import OneHotEncoder

 """
 
 encode_candidate_cols = df_all.columns.drop(
    ['StudyInstanceUID', 'ID', 'folds', 'labels', 'n_label', 'SOPInstanceUID', 'PatientID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID']
    + target_cols)
 
for c in encode_candidate_cols:
    if df_all[c].dtype == 'O' and c != "ImagePositionPatient":
        print(c)
        tmp = pd.concat([df_all[c], model_df_list_test[0][0][c]]).reset_index(drop=True)
        #tmp = df_all[c]
        tmp = pd.factorize(tmp)[0]
        n_label = tmp.max()+1
        ohe = OneHotEncoder(sparse=False)
        tmp = ohe.fit_transform(tmp.reshape(-1, 1))
        c_names = [c+"_{}".format(j) for j in range(n_label)]
        tmp = pd.DataFrame(tmp, columns=c_names)
        df_all = pd.concat([df_all, tmp[:len(df_all)]], axis=1).drop(columns=[c])
        for i in range(len(model_df_list_test)):
            for j in range(len(model_df_list_test[i])):
                model_df_list_test[i][j] = pd.concat([model_df_list_test[i][j], tmp[len(df_all):].reset_index(drop=True)], axis=1).drop(columns=[c])

    elif c == "ImagePositionPatient":
        # こいつだけ数が多すぎるのでlabel encoding
        print(c)
        tmp = pd.concat([df_all[c], model_df_list_test[0][0][c]]).reset_index(drop=True)
        tmp = pd.factorize(tmp)[0]
        df_all[c] = tmp[:len(df_all)]
        for i in range(len(model_df_list_test)):
            for j in range(len(model_df_list_test[i])):
                tmp_df = model_df_list_test[i][j]
                tmp_df[c] = tmp[len(df_all):]
                model_df_list_test[i][j] = tmp_df
"""

ImageOrientationPatient


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ImagePositionPatient
Modality


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


PhotometricInterpretation


...

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


WindowCenter


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


WindowWidth


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
# 6: target
# 30: 共通のパラメータ
# 507: 共通のパラメータ起因の特徴量
# 336: pred起因の特徴量

print(len(df_all.columns)), print(len(model_df_list_test[0][0].columns))

30+507+336, 6+30+507+336*7,  6+30+507+336*10

3407
377


(873, 2895, 3903)

In [ ]:
X_cols = df_all.columns.drop([c for c in df_all.columns if "pred" not in c]+target_cols)

df_all = df_all[list(X_cols)+target_cols+["ID", "folds", "StudyInstanceUID", "position"]]
for i in range(len(model_df_list_test)):
    for j in range(len(model_df_list_test[0])):
        cols = model_df_list_test[i][j].columns
        cols = [col for col in cols if col in list(X_cols)+["ID", "folds", "StudyInstanceUID", "position"]]
        model_df_list_test[i][j] = model_df_list_test[i][j][cols]

In [ ]:
n_models = len(model_df_list_test)
n_folds = len(model_df_list_test[0])
test_cols = set([])
for i in range(n_models):
    test_cols = test_cols | set(model_df_list_test[i][0].columns)

test_list = []
for j in range(n_folds):
    tmp_test_cols = test_cols
    for i in range(n_models):
        cols = model_df_list_test[i][j].columns
        use_cols = [col for col in cols if col in tmp_test_cols]
        if i == 0:
            df = model_df_list_test[i][j][use_cols]
        else:
            df = pd.merge(df, model_df_list_test[i][j][use_cols+["ID", ]], on="ID", how="outer")
        tmp_test_cols = tmp_test_cols - set(use_cols)
        print(len(tmp_test_cols))
    test_list.append(df)


In [ ]:
with open("df_all_preprcessed_10models.pkl", "wb") as f:
    pickle.dump(df_all, f, protocol=4)
with open("test_list_preprocessed_10models.pkl", "wb") as f:
    pickle.dump(test_list, f, protocol=4)
#with open("df_all_preprcessed_10models.pkl", "rb") as f:
#    df_all = pickle.load(f)
#with open("test_list_preprocessed_10models.pkl", "rb") as f:
#    test_list = pickle.load(f)


In [ ]:
X_cols = df_all.columns.drop(
    ['StudyInstanceUID', 'ID', 'folds', 'position']
    + target_cols)

# LGBM`

In [21]:
# 学習

import lightgbm as lgb
from sklearn.linear_model import Ridge, LogisticRegression

stack_preds = []
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          "num_leaves": 5,
          "max_bin": 256,
          "feature_fraction": 0.8,
          "verbosity": 0,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }
res = []
pred_test = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    tr = df_all.query('folds != {}'.format(i))
    va = df_all.query('folds == {}'.format(i))
    preds = pd.DataFrame(np.zeros([len(va), 6]), columns = [c + '_pred' for c in target_cols])
    for tar_col in target_cols:
        tr_D = lgb.Dataset(tr[X_cols], tr[tar_col])
        va_D = lgb.Dataset(va[X_cols], va[tar_col])
        clf = lgb.train(params, tr_D, 10000, valid_sets=va_D, verbose_eval=100, early_stopping_rounds=10)
        preds[tar_col + '_pred'] = clf.predict(va[X_cols])
        pred_test.append(clf.predict(test_list[i][X_cols]))
        print(log_loss(va[tar_col], preds[tar_col + '_pred']))
        print('-'*80)
        res.append(log_loss(va[tar_col], preds[tar_col + '_pred']))
    print('='*80)
    stack_preds.append(preds)

fold 1/5

Training until validation scores don't improve for 10 rounds
[100]	valid_0's binary_logloss: 0.101823
Early stopping, best iteration is:
[115]	valid_0's binary_logloss: 0.101566
0.10156623488761363
--------------------------------------------------------------------------------


KeyboardInterrupt: 

In [19]:
ids = test_list[0].ID
pred_test_array = np.zeros((5, pred_test[0].shape[0], 6))
for fold in range(5):
    for i, pred_row in enumerate(pred_test[fold*6:(fold+1)*6]):
        pred_test_array[fold, :, i] = pred_row

In [27]:
pred_test_array = np.mean(pred_test_array, axis=0)
pred_test_array.shape

(78363, 6)

In [28]:
label_to_num = {
    'any': 0,
    'epidural': 1,
    'subdural': 2,
    'subarachnoid': 3,
    'intraventricular': 4,
    'intraparenchymal': 5,
}
num_to_label = {v:k for k,v in label_to_num.items()}

IDs = {}
for id, pred in zip(ids, pred_test_array):
    for i, pred_each in enumerate(pred):
        label = num_to_label[i]
        ID = '%s_%s' % (id, label)
        IDs[ID] = pred_each

In [36]:
sub = pd.read_csv("../data/stage_1_sample_submission.csv")
sub["Label"] = sub.ID.map(IDs)
sub["Label"] = np.clip(sub.Label, 1e-15, 1-1e-15)
sub.loc[sub.Label.isnull(),'Label'] = sub.Label.min()

In [37]:
sub.to_csv("./lgbm_stacking_5models.csv", index=False)

In [34]:
sub

,ID,Label
0,ID_28fbab7eb_epidural,0.000193
1,ID_28fbab7eb_intraparenchymal,0.000202
2,ID_28fbab7eb_intraventricular,0.000089
3,ID_28fbab7eb_subarachnoid,0.000280
4,ID_28fbab7eb_subdural,0.000725
...,...,...
471265,ID_d47269109_intraparenchymal,0.000215
471266,ID_d47269109_intraventricular,0.000118
471267,ID_d47269109_subarachnoid,0.000462
471268,ID_d47269109_subdural,0.000760


# CNN

In [42]:
X_cols.shape

(2688,)

In [43]:
# 6: target
# 10: 共通の捨てるパラメータ
# 20: 共通の使うパラメータ
# 507: 共通のパラメータ起因の特徴量
# 336: pred起因の特徴量

print(len(df_all.columns)), print(len(model_df_list_test[0][0].columns)), print(len(test_list[0].iloc[0]))

# target+["ID", "folds", "StudyInstanceUID", "position"]+pred由来
6+4+336*8


2698
340
2692


2362

In [ ]:
columns_all = list(df_all.columns.drop(['StudyInstanceUID', 'ID', 'folds', 'position']))
print(len(columns_all))
columns_models = []
for model in models:
    columns_model = [c for c in df_all.columns if model in c]
    columns_models.append(columns_model)
    columns_all = [c for c in columns_all if c not in columns_model]
columns_intersection = set(columns_all) - set(target_cols)
print(len(columns_intersection), [len(columns_model) for columns_model in columns_models], len(target_cols))

In [ ]:
df_all = df_all.sort_values(["folds", 'StudyInstanceUID', 'position']).reset_index(drop=True)
for i in range(len(test_list)):
    test_list[i] = test_list[i].sort_values(['StudyInstanceUID', 'position']).reset_index(drop=True)

In [ ]:
X_train_list = []
#X_train_intersection_list = []
X_valid_list = []
#X_valid_intersection_list = []
for i in range(len(test_list)):
    df_train = df_all[df_all.folds!=i]
    tmp_train = np.empty((len(models), len(df_train), len(columns_models[0])))
    df_valid = df_all[df_all.folds==i]
    tmp_valid = np.empty((len(models), len(df_valid), len(columns_models[0])))
    for j in range(len(models)):
        tmp_train[j] = df_train[columns_models[j]]
        tmp_valid[j] = df_valid[columns_models[j]]
    X_train_list.append(tmp_train)
    #X_train_intersection_list.append(df_train[list(columns_intersection)].values)
    X_valid_list.append(tmp_valid)
    #X_valid_intersection_list.append(df_valid[list(columns_intersection)].values)

In [ ]:
X_train_list[0].shape

In [ ]:
# n_folds, n_models, n_samples, n_features
print(len(X_train_list), len(X_train_list[0]), len(X_train_list[0][0]), len(X_train_list[0][0][0]))
print(len(X_valid_list), len(X_valid_list[0]), len(X_valid_list[0][0]), len(X_valid_list[0][0][0]))

# n_folds, n_samples, n_features
#print(len(X_train_intersection_list), len(X_train_intersection_list[0]), len(X_train_intersection_list[0][0]))
#print(len(X_valid_intersection_list), len(X_valid_intersection_list[0]), len(X_valid_intersection_list[0][0]))

In [ ]:
train_batch_size_list = []
for i in range(5):
    train_batch_size_list.append(df_all[df_all.folds==i].groupby(["StudyInstanceUID"]).apply(lambda x: x.shape[0]).values)

test_batch_size = test_list[0].groupby(["StudyInstanceUID"]).apply(lambda x: x.shape[0]).values

In [ ]:
all_train_batch_counter = []

for i in range(5):
    train_batch_num = 0
    train_batch_counter = []
    for j, train_batch_size in enumerate(train_batch_size_list):
        if i != j:
            for train_batch_n in train_batch_size:
                train_batch_counter.extend([train_batch_num, ]*train_batch_n)
                train_batch_num += 1
    all_train_batch_counter.append(train_batch_counter)

all_valid_batch_counter = []
for i in range(5):
    valid_batch_num = 0
    valid_batch_counter = []
    for j in train_batch_size_list[i]:
        valid_batch_counter.extend([valid_batch_num, ]*j)
        valid_batch_num += 1
    all_valid_batch_counter.append(valid_batch_counter)


In [ ]:
test_batch_num = 0
test_batch_counter = []
for j in test_batch_size:
    test_batch_counter.extend([test_batch_num, ]*j)
    test_batch_num += 1


In [ ]:
y_train_list = []
y_valid_list = []
for i in range(5):
    df = df_all[df_all.folds!=i]
    y_train_list.append(df[target_cols].values)
    df = df_all[df_all.folds==i]
    y_valid_list.append(df[target_cols].values)


In [ ]:
# n_folds, n_samples, n_target
print(len(y_train_list), len(y_train_list[0]), len(y_train_list[0][0]))
print(len(y_valid_list), len(y_valid_list[0]), len(y_valid_list[0][0]))

In [ ]:
X_test = np.empty((len(test_list), len(models), len(test_list[0]), len(columns_models[0])))
X_test_list = []
#X_test_intersection_list = []
for i in range(len(test_list)):
    tmp_test = np.empty((len(models), len(test_list[0]), len(columns_models[0])))
    for j in range(len(models)):
        tmp_test[j] = test_list[i][columns_models[j]]
    X_test_list.append(tmp_test)
    #X_test_intersection_list.append(test_list[i][list(columns_intersection)].values)

In [ ]:
# n_folds, n_models, n_samples, n_features
print(len(X_test_list), len(X_test_list[0]), len(X_test_list[0][0]), len(X_test_list[0][0][0]))
# n_folds, n_samples, n_features
#print(len(X_test_intersection_list), len(X_test_intersection_list[0]), len(X_test_intersection_list[0][0]))

In [106]:
#dataset = (X_train_list, X_valid_list, y_train_list, y_valid_list, X_test_list)
#with open("dataset.pkl", "wb") as f:
#    pickle.dump(dataset, f, protocol=4)

In [ ]:
for i, X_train in enumerate(X_train_list):
    X_train_list[i] = X_train.transpose(1, 0, 2)
for i, X_valid in enumerate(X_valid_list):
    X_valid_list[i] = X_valid.transpose(1, 0, 2)
for i, X_test in enumerate(X_test_list):
    X_test_list[i] = X_test.transpose(1, 0, 2)

In [ ]:
"""
class StackingDataset(Dataset):
    def __init__(self, X, X_intersection, y):
        self.X = X
        self.X_intersection = X_intersection
        self.y = y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.X_intersection[idx]), torch.FloatTensor(self.y[idx])
    
class StackingDatasetTest(Dataset):
    def __init__(self, X, X_intersection):
        self.X = X
        self.X_intersection = X_intersection
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.X_intersection[idx])
"""
class StackingContinuousDataset(Dataset):
    def __init__(self, X, batch_num_list, y):
        self.X = X
        self.y = y
        self.batch_num_list = batch_num_list
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        if idx == 0:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), torch.FloatTensor(self.y[idx])
        elif idx ==  len(self.batch_num_list)-1:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1] and self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx, idx, idx]]), torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1]:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), torch.FloatTensor(self.y[idx])
        else:
            return torch.FloatTensor(self.X[[idx-1, idx, idx+1]]), torch.FloatTensor(self.y[idx])

class StackingContinuousDatasetTest(Dataset):
    def __init__(self, X, batch_num_list):
        self.X = X
        self.batch_num_list = batch_num_list
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        if idx == 0:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]])
        elif idx == len(self.batch_num_list)-1:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1] and self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx, idx, idx]])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1]:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]])
        elif self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]])
        else:
            return torch.FloatTensor(self.X[[idx-1, idx, idx+1]])

In [ ]:
X_train_list[0].shape

In [ ]:
"""
def get_loader(fold, mode):
    if mode == "train":
        train_loader = DataLoader(
            StackingDataset(X_train_list[fold], X_train_intersection_list[fold], y_train_list[fold]),
            shuffle=True,
            batch_size=128,
            num_workers=8,
        #    pin_memory=True,
        )
        return train_loader
    elif mode == "valid":
        valid_loader = DataLoader(
            StackingDataset(X_valid_list[fold], X_valid_intersection_list[fold], y_valid_list[fold]),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return valid_loader
    elif mode == "test":
        test_loader = DataLoader(
            StackingDatasetTest(X_test_list[fold], X_test_intersection_list[fold]),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return test_loader
    else:
        assert False
"""

def get_loader(fold, mode):
    if mode == "train":
        train_loader = DataLoader(
            StackingContinuousDataset(X_train_list[fold], all_train_batch_counter[fold], y_train_list[fold]),
            shuffle=True,
            batch_size=128,
            num_workers=8,
        #    pin_memory=True,
        )
        return train_loader
    elif mode == "valid":
        valid_loader = DataLoader(
            StackingContinuousDataset(X_valid_list[fold], all_valid_batch_counter[fold], y_valid_list[fold]),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return valid_loader
    elif mode == "test":
        test_loader = DataLoader(
            StackingContinuousDatasetTest(X_test_list[fold], test_batch_counter),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return test_loader
    else:
        assert False

In [ ]:
#np.random.seed(seed=100)
#swap_columns_idx = [np.random.permutation(np.arange(len(models))) for i in range(8)]


class StackingModel(nn.Module):
    def __init__(self):
        super(StackingModel, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(4, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
        )
        """
        self.dense_intersection_layer = nn.Sequential(
            nn.Linear(527, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
        )
        """
        self.dense_concat_layer = nn.Sequential(
            nn.Linear(21504, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, 6),

#            nn.ReLU(inplace=True),
#            nn.Linear(1024, 128),
#            nn.ReLU(inplace=True),
#            nn.Linear(128, 6),
        )
        
    def forward(self, x):
        #x, x_intersection = xs
        #for i in range(8):
        #    if i == 0:
        #        shuffled_xs = x[:, :, swap_columns_idx[i], :]
        #    else:
        #        shuffled_xs = torch.cat((shuffled_xs, x[:, :, swap_columns_idx[i], :]), 1)
        #x = self.cnn_layer(shuffled_xs)
        x = self.cnn_layer(x)
        x = x.view(x.shape[0], -1)
        #x_intersection = self.dense_intersection_layer(x_intersection)
        #x_intersection = x_intersection.view(x.shape[0], -1)
        #out = torch.cat((x, x_intersection), 1)
        x = self.dense_concat_layer(x)
        #x = torch.clamp(x, min=1e-15, max=1-1e-15)
        return x

In [ ]:
print(StackingModel())

In [ ]:
pred_test = []
pred_valid = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    tr = get_loader(i, "train")
    va = get_loader(i, "valid")
    te = get_loader(i, "test")
    
    model = StackingModel().cuda()
    optimizer = torch.optim.Adam(model.parameters(), 2e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1, 2, 3, 4, 6, 9, 12, 15], gamma=0.5)
    criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2, 1, 1, 1, 1, 1]).cuda())
    
    n_epoch = 1
    pred_valid_fold = []
    for epoch in range(n_epoch):
        model.train()
        losses = []
        for x, y in tr:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.item())

        losses_valid = []
        with torch.no_grad():
            model.eval()
            for x, y in va:                
                pred = model(x.cuda())
                loss = criterion(pred, y.cuda())
                losses_valid.append(loss.item())
                if epoch == n_epoch-1:
                    pred_valid_fold.append(pred)
            print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
        scheduler.step()

    with torch.no_grad():
        model.eval()
        pred_test_fold = []
        for x in te:            
            pred = model(x.cuda())
            pred_test_fold.append(pred)
    pred_test.append(pred_test_fold)
    pred_valid.append(pred_valid_fold)

In [115]:
np.where(np.array(losses)==32931776.0)

(array([2989]),)

In [116]:
losses[2989]

32931776.0

In [ ]:
"""
def get_channel_data(x_batch):
    new_batch = np.empty((x_batch.shape[0], 3, x_batch.shape[1], x_batch.shape[2]))
    new_batch[0] = x_batch[[0, 0, 1]]
    new_batch[1:-1] = np.array([x_batch[[i-1, i, i+1]] for i in range(1, len(x_batch)-1)])
    new_batch[-1] = x_batch[[-2, -1, -1]]
    return new_batch
"""

In [158]:
"""
pred_test = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    #tr = get_loader(i, "train")
    #va = get_loader(i, "valid")
    #te = get_loader(i, "test")
    
    model = StackingModel().cuda()
    optimizer = torch.optim.Adam(model.parameters(), 4e-5)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 6, 9, 12, 15], gamma=0.5)
    criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2, 1, 1, 1, 1, 1]).cuda())
    
    for epoch in range(20):
        model.train()
        losses = []
        
        #for x, x_intersection, y in tr:
        for j in range(5):
            if j != i:
                n_batch_start = 0
                for batch_size in train_batch_size_list[j]:
                    n_batch_end = n_batch_start + batch_size
                    x = X_train_list[j][n_batch_start:n_batch_end]
                    x = get_channel_data(x)
                    y = y_train_list[j][n_batch_start:n_batch_end]
                    

                    #pred = model((x.cuda(), x_intersection.cuda()))
                    pred = model(torch.from_numpy(x).float().cuda())
                    loss = criterion(pred, torch.from_numpy(y).float().cuda())
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    losses.append(loss.item())

        losses_valid = []
        with torch.no_grad():
            model.eval()
            val_pred = np.zeros((1, 6))
            val_target = np.zeros((1, 6))
            
            #for x, x_intersection, y in va:
            n_batch_start = 0
            for batch_size in train_batch_size_list[i]:
                n_batch_end = n_batch_start + batch_size
                x = X_train_list[i][n_batch_start:n_batch_end]
                x = get_channel_data(x)
                y = y_train_list[i][n_batch_start:n_batch_end]
                
                #pred = model((x.cuda(), x_intersection.cuda()))
                pred = model(torch.from_numpy(x).float().cuda())
                loss = criterion(pred, torch.from_numpy(y).float().cuda())
                losses_valid.append(loss.item())
                val_pred = np.concatenate((val_pred, pred.cpu()), axis=0)
                val_target = np.concatenate((val_target, y), axis=0)
            print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
            val_pred = val_pred[1:]
            val_target = val_target[1:]
            logloss_list = []
            for i in range(6):
                logloss_list.append(log_loss(val_target[:, i], val_pred[:, i]))
            print("logloss : {}".format(logloss_list))
        scheduler.step()

    with torch.no_grad():
        model.eval()
        
        #for x, x_intersection in te:
        n_batch_start = 0
        for batch_size in test_batch_size:
            n_batch_end = n_batch_start + batch_size
            x = X_train_list[i][n_batch_start:n_batch_end]
            x = get_channel_data(x)
            y = y_train_list[i][n_batch_start:n_batch_end]
            
            #pred = model((x.cuda(), x_intersection.cuda()))
            pred = model(torch.from_numpy(x).float().cuda())
            pred_test.append(pred)
"""

fold 1/5

Epoch[0] Train: 0.0015340156306613555 Valid: 19.1010989856264


ValueError: y_true contains only one label (0.0). Please provide the true labels explicitly through the labels argument.

In [86]:
class StackingModel2(nn.Module):
    def __init__(self):
        super(StackingModel2, self).__init__()
        self.dense_layer = nn.Sequential(
            nn.Linear(7056, 1000),
            nn.Tanh(),
            nn.Linear(1000, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 6),
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 7056)
        out = self.dense_layer(x)
        return out

In [87]:
model = StackingModel2().cuda()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2,1,1,1,1,1]).cuda())

In [88]:
print(model)

StackingModel2(
  (dense_layer): Sequential(
    (0): Linear(in_features=7056, out_features=1000, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1000, out_features=200, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=200, out_features=6, bias=True)
  )
)


In [89]:
pred_test = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    tr = get_loader(i, "train")
    va = get_loader(i, "valid")
    te = get_loader(i, "test")
    for epoch in range(10):
        print(epoch)
        model.train()
        losses = []
        for x, y in tr:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            loss.backward()
            optimizer.step()
            
            optimizer.zero_grad()
            losses.append(loss.item())

        losses_valid = []
        with torch.no_grad():
            model.eval()
            for x, y in va:
                pred = model(x.cuda())
                loss = criterion(pred, y.cuda())
                losses_valid.append(loss.item())
            print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))

    with torch.no_grad():
        model.eval()
        for x in te:
            pred = model(x.cuda())
            pred_test.append(pred)

fold 1/5

0
Epoch[0] Train: nan Valid: nan
1


KeyboardInterrupt: 

In [24]:
model.train()
for epoch in range(10):
    losses = []
    for x, y in train_loader:
        pred = model(x.cuda())
        loss = criterion(pred, y.cuda())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
    losses_valid = []
    with torch.no_grad():
        model.eval()
        for x, y in valid_loader:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            losses_valid.append(loss.item())
    print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
    model.train()

Epoch[0] Train: 0.24050997862371834 Valid: 0.16861760471272105
Epoch[1] Train: 0.1602081820329327 Valid: 0.15870382193163152
Epoch[2] Train: 0.15492199151933594 Valid: 0.15616315897582597
Epoch[3] Train: 0.1529936831932124 Valid: 0.15505525547982626
Epoch[4] Train: 0.15195317921713639 Valid: 0.1544597020216343
Epoch[5] Train: 0.15126592678459821 Valid: 0.1540926653493679
Epoch[6] Train: 0.15077115383440193 Valid: 0.15387332139408771
Epoch[7] Train: 0.15038474744384797 Valid: 0.15371600970739627
Epoch[8] Train: 0.1500640829005014 Valid: 0.1535945014445154
Epoch[9] Train: 0.14979854242246923 Valid: 0.15351682496150487


In [ ]:
y

In [ ]:
criterion(pred, y.cuda()).item()

In [32]:
disp_full(model_dfs[0])

,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred,folds,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,ID,StudyInstanceUID,PositionOrd,any_min,any_max,any_mean,any_std,epidural_min,epidural_max,epidural_mean,epidural_std,subdural_min,subdural_max,subdural_mean,subdural_std,subarachnoid_min,subarachnoid_max,subarachnoid_mean,subarachnoid_std,intraventricular_min,intraventricular_max,intraventricular_mean,intraventricular_std,intraparenchymal_min,intraparenchymal_max,intraparenchymal_mean,intraparenchymal_std,any_diff,any_div,any_scaled,epidural_diff,epidural_div,epidural_scaled,subdural_diff,subdural_div,subdural_scaled,subarachnoid_diff,subarachnoid_div,subarachnoid_scaled,intraventricular_diff,intraventricular_div,intraventricular_scaled,intraparenchymal_diff,intraparenchymal_div,intraparenchymal_scaled,any_3roll,epidural_3roll,subdural_3roll,subarachnoid_3roll,intraventricular_3roll,intraparenchymal_3roll,any_5roll,epidural_5roll,subdural_5roll,subarachnoid_5roll,intraventricular_5roll,intraparenchymal_5roll,any_3rolldiff,epidural_3rolldiff,subdural_3rolldiff,subarachnoid_3rolldiff,intraventricular_3rolldiff,intraparenchymal_3rolldiff,any_5rolldiff,epidural_5rolldiff,subdural_5rolldiff,subarachnoid_5rolldiff,intraventricular_5rolldiff,intraparenchymal_5rolldiff,any_3rolldiv,epidural_3rolldiv,subdural_3rolldiv,subarachnoid_3rolldiv,intraventricular_3rolldiv,intraparenchymal_3rolldiv,any_5rolldiv,epidural_5rolldiv,subdural_5rolldiv,subarachnoid_5rolldiv,intraventricular_5rolldiv,intraparenchymal_5rolldiv
0,0.000051,2.000906e-06,0.000009,0.000019,4.938827e-06,0.000002,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_3a422b8d7,ID_00047d6503,0.025000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.039541,0.001300,-0.671077,-0.008222,0.000243,-0.505105,-0.027808,0.000308,-0.616448,-0.005320,0.003493,-0.775595,-0.001138,0.004320,-0.428411,-0.004934,0.000506,-0.487321,0.000057,0.000003,0.000012,0.000018,0.000005,0.000004,0.000258,0.000023,0.000047,0.000122,0.000035,0.000029,0.000005,0.000001,0.000004,-6.081101e-07,-1.911224e-07,0.000001,0.000206,0.000021,0.000038,0.000103,0.000030,0.000026,1.103592,1.640546,1.446989,0.967389,0.961302,1.515170,5.008629,11.420777,5.434845,6.517519,6.999029,11.482114
1,0.000062,4.564251e-06,0.000016,0.000017,4.556583e-06,0.000005,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_490b10d5a,ID_00047d6503,0.050000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.039531,0.001569,-0.670896,-0.008220,0.000555,-0.504947,-0.027800,0.000583,-0.616278,-0.005321,0.003265,-0.775772,-0.001139,0.003985,-0.428555,-0.004931,0.001028,-0.487066,0.000258,0.000023,0.000047,0.000122,0.000035,0.000029,0.001391,0.000083,0.000374,0.000589,0.000232,0.000268,0.000196,0.000018,0.000030,1.041049e-04,3.001041e-05,0.000024,0.001329,0.000079,0.000358,0.000571,0.000227,0.000263,4.149021,5.006713,2.869538,6.972257,7.586166,5.655266,22.391411,18.277673,23.067760,33.769063,50.842242,52.751340
2,0.000660,6.199053e-05,0.000115,0.000329,9.420557e-05,0.000079,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_be2a0ca1c,ID_00047d6503,0.075000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.038933,0.016661,-0.660755,-0.008162,0.007537,-0.501420,-0.027702,0.004129,-0.614092,-0.005010,0.061539,-0.730417,-0.001049,0.082396,-0.394817,-0.004858,0.015910,-0.479810,0.001837,0.000111,0.000496,0.000779,0.000307,0.000356,0.003206,0.000142,0.000703,0.001872,0.000482,0.000533,0.001178,0.000049,0.000381,4.501109e-04,2.13

In [34]:
csv = pd.read_csv("../data/stage_1_train.csv")
n_train = 4045572//6

name = []
value = []
for i in tqdm(range(n_train), total=n_train):
    name.append(csv.iloc[i*6].ID.replace("_epidural", ""))
    value.append(csv.iloc[i*6:(i+1)*6].Label.to_list())

100%|██████████| 674262/674262 [04:31<00:00, 2481.66it/s]


In [86]:
csv_df = pd.DataFrame(value, columns=["epidural", "intraparenchymal", "intraventricular", "subarachnoid", "subdural", "any"])
csv_df["ID"] = list(map(lambda x: x.replace("_epidural", ""), name))
csv_df = csv_df[["ID", "any", "epidural", "subdural", "subarachnoid", "intraventricular", "intraparenchymal"]]
display(csv_df)

,ID,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal
0,ID_63eb1e259,0,0,0,0,0,0
1,ID_2669954a7,0,0,0,0,0,0
2,ID_52c9913b1,0,0,0,0,0,0
3,ID_4e6ff6126,0,0,0,0,0,0
4,ID_7858edd88,0,0,0,0,0,0
...,...,...,...,...,...,...,...
674257,ID_f737f4cc1,0,0,0,0,0,0
674258,ID_4c92d70b5,0,0,0,0,0,0
674259,ID_a9797cb3a,0,0,0,0,0,0
674260,ID_9375f67bd,0,0,0,0,0,0
